In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# 회귀분석
from statsmodels.formula.api import ols
# qqplot, 회귀분석의 상수항 추가
from statsmodels.api import qqplot, add_constant
# 선형 모델 formula(y ~ X1 + X2 + ...)
import statsmodels.formula.api as smf
# 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor
# 선형회귀모형
from sklearn.linear_model import LinearRegression
# 변수 선택법(후진제거법)
from sklearn.feature_selection import RFE
# Scale 변환->표준화 회귀계수 산출
from sklearn.preprocessing import StandardScaler
# 평가함수
from statsmodels.tools.eval_measures import rmse
# jupyter notebook 사용 시 그래프 자동 출력 옵션
import matplotlib
from sklearn.tree import export_graphviz
import graphviz

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
%matplotlib inline


In [6]:
import warnings
warnings.filterwarnings(action='ignore')

In [7]:
import statsmodels.api as sm

In [2]:
# 한글 폰트(맑은 고딕) 적용
matplotlib.rc("font", family = "AppleGothic")

In [3]:
import os
# graphviz path 설정
# os.environ["PATH"] += os.pathsep + "C:/Program Files (x86)/Graphviz2.38/bin/"

In [4]:
# sklearn.tree 의 DecisionTreeClassifier: 분류의사결정나무
from sklearn.tree import DecisionTreeClassifier

# sklearn.tree의 export_graphviz: graphviz 패키지가 사용할 수 있는 .dot 확장자 파일로 저장
from sklearn.tree import export_graphviz

# graphviz import: 의사결정 나무 모델 시각화  .dot 확장자 파일 불러오는 패키지
import graphviz

In [32]:
df_raw = pd.read_csv("DataSet_final.csv")

In [8]:
# 더미 변수화
df_raw[['Ox_Chamber']]=df_raw[['Ox_Chamber']].astype(str)
df_raw[['photo_soft_Chamber']]=df_raw[['photo_soft_Chamber']].astype(str)
df_raw[['lithography_Chamber']]=df_raw[['lithography_Chamber']].astype(str)
df_raw[['Etching_Chamber']]=df_raw[['Etching_Chamber']].astype(str)
df_raw[['Chamber_Num']]=df_raw[['Chamber_Num']].astype(str)
df_raw[['Wavelength']]=df_raw[['Wavelength']].astype(str)

# wet, dry 구분
df_raw.loc[df_raw['type']=='dry','type']=0
df_raw.loc[df_raw['type']=='wet','type']=1

# Vapor / H2O, O2 구분
df_raw.loc[df_raw['Vapor']=='H2O','Vapor']=0
df_raw.loc[df_raw['Vapor']=='O2','Vapor']=1

# Vapor / H2O, O2 구분
df_raw.loc[df_raw['Vapor']=='H2O','Vapor']=0
df_raw.loc[df_raw['Vapor']=='O2','Vapor']=1

# Wavelength 365, 405, 436
df_raw.loc[df_raw['Wavelength']=='365','Wavelength']=0
df_raw.loc[df_raw['Wavelength']=='405','Wavelength']=1
df_raw.loc[df_raw['Wavelength']=='436','Wavelength']=2

In [30]:
df_list_before=['type', 'Temp_OXid', 'Vapor', 'ppm','Pressure', 'Oxid_time', 'thickness',
        'resist_target', 'N2_HMDS', 'pressure_HMDS', 'temp_HMDS','temp_HMDS_bake', 'time_HMDS_bake', 'spin1', 'spin2', 'spin3','photoresist_bake', 'temp_softbake', 'time_softbake',
        'Line_CD','Wavelength', 'Resolution', 'Energy_Exposure',
        'Thin F4','Thin F2','Thin F3','Thin F1', 'Temp_Etching','Source_Power', 'Selectivity',
        'Flux60s', 'Flux90s', 'Flux160s', 'Flux480s','Flux840s', 'input_Energy', 'Temp_implantation','Furance_Temp', 'RTA_Temp']

In [38]:
df_list = pd.DataFrame()

In [48]:
df_list  = df_raw[['type', 'Temp_OXid', 'Vapor', 'ppm','Pressure', 'Oxid_time','thickness']]
                   

In [49]:
df_list

,type,Temp_OXid,Vapor,ppm,Pressure,Oxid_time,thickness
0,dry,956.156908,H2O,33.41,0.218,144,713.383
1,dry,872.064192,O2,22.14,0.217,224,714.028
2,dry,1090.050665,O2,41.20,0.206,82,703.101
3,dry,872.534533,O2,22.23,0.211,218,707.850
4,dry,1061.165672,H2O,31.85,0.204,162,696.610
...,...,...,...,...,...,...,...
825,wet,1275.182502,H2O,45.10,0.214,21,715.498
826,dry,1045.950206,O2,44.94,0.218,89,715.913
827,wet,1107.041866,O2,32.59,0.212,118,719.257
828,wet,1259.826425,O2,37.72,0.206,145,720.692


In [50]:
df_list.corr()

,Temp_OXid,ppm,Pressure,Oxid_time,thickness
Temp_OXid,1.000000,0.825572,0.008435,-0.807454,0.069695
ppm,0.825572,1.000000,0.007888,-0.814348,0.087249
Pressure,0.008435,0.007888,1.000000,0.064961,0.365254
Oxid_time,-0.807454,-0.814348,0.064961,1.000000,-0.041462
thickness,0.069695,0.087249,0.365254,-0.041462,1.000000


In [51]:
df_list1  = df_raw[['resist_target', 'N2_HMDS', 'pressure_HMDS', 'temp_HMDS','temp_HMDS_bake', 'time_HMDS_bake', 'spin1', 'spin2', 'spin3','photoresist_bake', 'temp_softbake', 'time_softbake']]

In [52]:
df_list1.corr()

,resist_target,N2_HMDS,pressure_HMDS,temp_HMDS,temp_HMDS_bake,time_HMDS_bake,spin1,spin2,spin3,photoresist_bake,temp_softbake,time_softbake
resist_target,1.000000,-0.099798,0.011119,-0.078626,-0.295366,0.078941,-0.323618,-0.256427,-0.349212,-0.285165,-0.311860,0.092130
N2_HMDS,-0.099798,1.000000,-0.072116,0.025839,0.036755,-0.015872,0.039318,0.028850,0.089896,0.054529,0.047231,-0.009258
pressure_HMDS,0.011119,-0.072116,1.000000,-0.036956,-0.021443,-0.010290,-0.063601,0.041027,-0.059957,-0.022904,-0.026142,0.029170
temp_HMDS,-0.078626,0.025839,-0.036956,1.000000,-0.034149,-0.008972,0.016052,-0.032589,-0.006724,0.030123,-0.017025,-0.036400
temp_HMDS_bake,-0.295366,0.036755,-0.021443,-0.034149,1.000000,-0.003525,0.499668,0.469701,0.517695,0.500693,0.424054,-0.039482
time_HMDS_bake,0.078941,-0.015872,-0.010290,-0.008972,-0.003525,1.000000,0.009308,-0.020008,0.005027,0.006232,0.028497,0.091181
spin1,-0.323618,0.039318,-0.063601,0.016052,0.499668,0.009308,1.000000,0.447329,0.491017,0.432674,0.451874,-0.045584
spin2,-0.256427,0.028850,0.041027,-0.032589,0.469701,-0.020008,0.447329,1.000000,0.448873,0.443041,0.457386,-0.002670
spin3,-0.349212,0.089896,-0.059957,-0.006724,0.517695,0.005027,0.491017,0.448873,1.000000,0.497817,0.456784,0.009546
photoresist_bake,-0.285165,0.054529,-0.022904,0.030123,0.500693,0.006232,0.432674,0.443041,0.497817,1.000000,0.426258,-0.040254


In [54]:
df_list2  = df_raw[['Line_CD','Wavelength', 'Resolution', 'Energy_Exposure']]

In [55]:
df_list2.corr()

,Line_CD,Wavelength,Resolution,Energy_Exposure
Line_CD,1.000000,0.002317,-0.053647,0.229850
Wavelength,0.002317,1.000000,0.545510,0.356712
Resolution,-0.053647,0.545510,1.000000,0.324039
Energy_Exposure,0.229850,0.356712,0.324039,1.000000


In [56]:
 df_list3 = df_raw[['Thin F4','Thin F2','Thin F3','Thin F1', 'Temp_Etching','Source_Power', 'Selectivity']]

In [57]:
df_list3.corr()

,Thin F4,Thin F2,Thin F3,Thin F1,Temp_Etching,Source_Power,Selectivity
Thin F4,1.000000,0.364260,0.352463,0.093366,-0.065093,0.015035,0.041381
Thin F2,0.364260,1.000000,0.287442,0.119891,0.096205,0.066114,0.060307
Thin F3,0.352463,0.287442,1.000000,0.065592,0.001559,0.027701,0.007527
Thin F1,0.093366,0.119891,0.065592,1.000000,0.084990,0.009327,0.006091
Temp_Etching,-0.065093,0.096205,0.001559,0.084990,1.000000,0.092630,0.090959
Source_Power,0.015035,0.066114,0.027701,0.009327,0.092630,1.000000,0.536643
Selectivity,0.041381,0.060307,0.007527,0.006091,0.090959,0.536643,1.000000


In [58]:
 df_list4 = df_raw[['Flux60s', 'Flux90s', 'Flux160s', 'Flux480s','Flux840s', 'input_Energy', 'Temp_implantation','Furance_Temp', 'RTA_Temp']]

In [59]:
df_list4.corr()

,Flux60s,Flux90s,Flux160s,Flux480s,Flux840s,input_Energy,Temp_implantation,Furance_Temp,RTA_Temp
Flux60s,1.000000,0.301364,0.239195,0.024792,0.006786,-0.003872,0.090685,-0.015689,0.084451
Flux90s,0.301364,1.000000,0.408259,0.088709,-0.130283,-0.205340,-0.022557,-0.034959,-0.050628
Flux160s,0.239195,0.408259,1.000000,0.067435,-0.157600,-0.260001,-0.035919,-0.058464,-0.037230
Flux480s,0.024792,0.088709,0.067435,1.000000,-0.043841,-0.079693,0.033653,0.002239,-0.080664
Flux840s,0.006786,-0.130283,-0.157600,-0.043841,1.000000,0.129560,0.062106,-0.013379,0.098927
input_Energy,-0.003872,-0.205340,-0.260001,-0.079693,0.129560,1.000000,0.054818,0.081157,0.075981
Temp_implantation,0.090685,-0.022557,-0.035919,0.033653,0.062106,0.054818,1.000000,0.036876,0.550257
Furance_Temp,-0.015689,-0.034959,-0.058464,0.002239,-0.013379,0.081157,0.036876,1.000000,0.023395
RTA_Temp,0.084451,-0.050628,-0.037230,-0.080664,0.098927,0.075981,0.550257,0.023395,1.000000


In [62]:
df_raw.columns

Index(['Unnamed: 0', 'Ox_Chamber', 'process', 'type', 'Temp_OXid', 'Vapor',
       'ppm', 'Pressure', 'Oxid_time', 'thickness', 'No_Die',
       'photo_soft_Chamber', 'process 2', 'resist_target', 'N2_HMDS',
       'pressure_HMDS', 'temp_HMDS', 'temp_HMDS_bake', 'time_HMDS_bake',
       'spin1', 'spin2', 'spin3', 'photoresist_bake', 'temp_softbake',
       'time_softbake', 'lithography_Chamber', 'Line_CD', 'Lamp', 'UV_type',
       'Wavelength', 'Resolution', 'Energy_Exposure', 'Etching_Chamber',
       'Process 3', 'Thin F4', 'Thin F3', 'Thin F2', 'Thin F1', 'Temp_Etching',
       'Material', 'Source_Power', 'Selectivity', 'Chamber_Num', 'process4',
       'Flux60s', 'Flux90s', 'Flux160s', 'Flux480s', 'Flux840s',
       'input_Energy', 'Current', 'Temp_implantation', 'Furance_Temp',
       'RTA_Temp', 'Lot_Num', 'Wafer_Num', 'Datetime', 'Target',
       'Error_message', 'Wafer_map', 'Path', 'Quality'],
      dtype='object')

In [91]:
df_list5 =  df_raw[[ 'Temp_OXid',
       'ppm', 'Pressure', 'Oxid_time', 'thickness', 
        'resist_target', 'N2_HMDS',
       'pressure_HMDS', 'temp_HMDS', 'temp_HMDS_bake', 'time_HMDS_bake',
       'spin1', 'spin2', 'spin3', 'photoresist_bake', 'temp_softbake',
       'time_softbake','Line_CD', 
        'Resolution', 'Energy_Exposure', 
        'Thin F4', 'Thin F3', 'Thin F2', 'Thin F1', 'Temp_Etching',
        'Source_Power', 'Selectivity', 
       'Flux60s', 'Flux90s', 'Flux160s', 'Flux480s', 'Flux840s',
       'input_Energy', 'Temp_implantation', 'Furance_Temp',
       'RTA_Temp', 
        'Target']]

In [92]:
pd.set_option('display.max_columns', None)

In [93]:
df_list5.corr().round(3)

,Temp_OXid,ppm,Pressure,Oxid_time,thickness,resist_target,N2_HMDS,pressure_HMDS,temp_HMDS,temp_HMDS_bake,time_HMDS_bake,spin1,spin2,spin3,photoresist_bake,temp_softbake,time_softbake,Line_CD,Resolution,Energy_Exposure,Thin F4,Thin F3,Thin F2,Thin F1,Temp_Etching,Source_Power,Selectivity,Flux60s,Flux90s,Flux160s,Flux480s,Flux840s,input_Energy,Temp_implantation,Furance_Temp,RTA_Temp,Target
Temp_OXid,1.000,0.826,0.008,-0.807,0.070,0.140,-0.113,0.012,0.014,-0.182,0.024,-0.189,-0.186,-0.189,-0.193,-0.135,0.042,0.001,-0.584,-0.376,0.058,0.049,0.143,0.084,0.070,-0.409,-0.372,-0.047,0.035,0.037,0.038,-0.050,-0.037,-0.388,-0.389,-0.385,-0.071
ppm,0.826,1.000,0.008,-0.814,0.087,0.151,-0.111,0.042,0.001,-0.195,0.052,-0.202,-0.196,-0.229,-0.199,-0.128,0.040,0.023,-0.611,-0.366,0.098,0.066,0.168,0.088,0.057,-0.399,-0.381,-0.044,0.029,0.044,0.047,-0.045,-0.018,-0.374,-0.366,-0.387,-0.047
Pressure,0.008,0.008,1.000,0.065,0.365,-0.097,0.161,-0.034,0.076,-0.055,-0.011,-0.127,-0.173,-0.073,-0.101,-0.166,-0.087,0.048,-0.129,0.085,-0.222,-0.108,0.027,0.078,0.353,0.125,0.124,-0.001,-0.251,-0.288,-0.072,0.154,0.357,0.145,-0.087,0.156,0.156
Oxid_time,-0.807,-0.814,0.065,1.000,-0.041,-0.143,0.114,-0.049,-0.030,0.201,-0.012,0.164,0.132,0.165,0.189,0.074,-0.046,-0.019,0.571,0.353,-0.081,-0.036,-0.151,-0.108,-0.048,0.394,0.408,0.042,-0.055,-0.066,-0.043,0.049,0.046,0.410,0.338,0.384,0.079
thickness,0.070,0.087,0.365,-0.041,1.000,-0.018,0.035,-0.059,0.055,-0.110,-0.050,-0.131,-0.211,-0.135,-0.125,-0.231,-0.024,0.062,-0.112,0.074,-0.063,-0.031,0.107,0.120,0.377,0.134,0.062,-0.043,-0.220,-0.173,0.000,0.118,0.219,0.105,-0.185,0.126,0.165
resist_target,0.140,0.151,-0.097,-0.143,-0.018,1.000,-0.100,0.011,-0.079,-0.295,0.079,-0.324,-0.256,-0.349,-0.285,-0.312,0.092,0.158,-0.117,-0.015,0.300,0.152,0.234,0.071,-0.040,-0.151,-0.077,-0.014,0.070,0.071,0.009,-0.028,-0.177,-0.099,-0.113,-0.097,0.282
N2_HMDS,-0.113,-0.111,0.161,0.114,0.035,-0.100,1.000,-0.072,0.026,0.037,-0.016,0.039,0.029,0.090,0.055,0.047,-0.009,-0.007,0.071,0.019,-0.127,-0.133,-0.115,-0.037,0.015,0.043,0.010,-0.219,-0.242,-0.257,-0.012,0.089,0.097,0.009,0.090,0.055,-0.034
pressure_HMDS,0.012,0.042,-0.034,-0.049,-0.059,0.011,-0.072,1.000,-0.037,-0.021,-0.010,-0.064,0.041,-0.060,-0.023,-0.026,0.029,0.050,-0.062,-0.023,0.028,0.047,0.020,-0.045,-0.053,-0.027,-0.028,-0.018,0.031,-0.007,0.020,0.012,-0.011,0.017,-0.036,-0.016,-0.023
temp_HMDS,0.014,0.001,0.076,-0.030,0.055,-0.079,0.026,-0.037,1.000,-0.034,-0.009,0.016,-0.033,-0.007,0.030,-0.017,-0.036,-0.034,0.001,0.007,-0.034,-0.066,0.004,0.026,0.035,0.005,0.029,0.003,-0.016,-0.118,-0.004,0.009,0.029,0.018,-0.017,-0.019,0.006
temp_HMDS_bake,-0.182,-0.195,-0.055,0.201,-0.110,-0.295,0.037,-0.021,-0.034,1.000,-0.004,0.500,0.470,0.518,0.501,0.424,-0.039,-0.106,0.119,0.153,-0.151,-0.093,-0.203,-0.127,-0.131,0.164,0.108,0.063,0.015,-0.046,-0.037,0.051,0.048,0.128,0.149,0.163,-0.178


In [94]:
df_raw['Spin4'] = df_raw['spin1']+df_raw['spin2']+df_raw['spin3']

In [95]:
df_raw['Spin4']

0      9593.131
1      9653.002
2      9585.290
3      9684.676
4      9529.104
         ...   
825    9703.750
826    9493.240
827    9508.243
828    9500.385
829    9450.028
Name: Spin4, Length: 830, dtype: float64

In [96]:
df_raw.corr().round(3)

,Unnamed: 0,Ox_Chamber,Temp_OXid,ppm,Pressure,Oxid_time,thickness,photo_soft_Chamber,resist_target,N2_HMDS,pressure_HMDS,temp_HMDS,temp_HMDS_bake,time_HMDS_bake,spin1,spin2,spin3,photoresist_bake,temp_softbake,time_softbake,lithography_Chamber,Line_CD,Wavelength,Resolution,Energy_Exposure,Etching_Chamber,Thin F4,Thin F3,Thin F2,Thin F1,Temp_Etching,Source_Power,Selectivity,Chamber_Num,Flux60s,Flux90s,Flux160s,Flux480s,Flux840s,input_Energy,Current,Temp_implantation,Furance_Temp,RTA_Temp,Lot_Num,Wafer_Num,Target,Path,Quality,Spin4
Unnamed: 0,1.000,0.943,0.024,-0.004,0.004,-0.024,-0.001,0.326,-0.053,0.004,0.028,0.030,0.034,-0.045,0.015,0.010,0.050,0.005,0.046,0.012,0.077,-0.047,-0.043,-0.036,0.047,0.027,-0.022,-0.031,0.027,0.031,0.030,-0.025,-0.018,0.027,0.072,0.022,-0.002,0.002,-0.057,-0.017,0.0,0.004,-0.017,0.020,0.024,-0.006,-0.030,0.971,-0.015,0.037
Ox_Chamber,0.943,1.000,0.016,-0.009,0.004,-0.024,0.026,0.013,-0.049,0.009,0.022,0.009,0.040,-0.046,0.017,0.012,0.050,-0.004,0.042,0.038,-0.036,-0.040,-0.042,-0.037,0.047,0.004,-0.027,-0.028,0.026,0.021,0.046,-0.024,-0.004,0.004,0.056,0.007,0.003,0.002,-0.043,-0.014,-0.0,0.011,-0.015,0.023,0.017,-0.011,-0.026,0.995,-0.017,0.039
Temp_OXid,0.024,0.016,1.000,0.826,0.008,-0.807,0.070,0.011,0.140,-0.113,0.012,0.014,-0.182,0.024,-0.189,-0.186,-0.189,-0.193,-0.135,0.042,0.020,0.001,-0.523,-0.584,-0.376,0.014,0.058,0.049,0.143,0.084,0.070,-0.409,-0.372,0.014,-0.047,0.035,0.037,0.038,-0.050,-0.037,-0.0,-0.388,-0.389,-0.385,0.132,0.036,-0.071,0.017,-0.115,-0.222
ppm,-0.004,-0.009,0.826,1.000,0.008,-0.814,0.087,0.018,0.151,-0.111,0.042,0.001,-0.195,0.052,-0.202,-0.196,-0.229,-0.199,-0.128,0.040,-0.022,0.023,-0.514,-0.611,-0.366,0.005,0.098,0.066,0.168,0.088,0.057,-0.399,-0.381,0.005,-0.044,0.029,0.044,0.047,-0.045,-0.018,0.0,-0.374,-0.366,-0.387,0.109,0.037,-0.047,-0.008,-0.082,-0.253
Pressure,0.004,0.004,0.008,0.008,1.000,0.065,0.365,0.003,-0.097,0.161,-0.034,0.076,-0.055,-0.011,-0.127,-0.173,-0.073,-0.101,-0.166,-0.087,-0.006,0.048,-0.054,-0.129,0.085,0.020,-0.222,-0.108,0.027,0.078,0.353,0.125,0.124,0.020,-0.001,-0.251,-0.288,-0.072,0.154,0.357,0.0,0.145,-0.087,0.156,0.550,0.012,0.156,0.004,0.151,-0.139
Oxid_time,-0.024,-0.024,-0.807,-0.814,0.065,1.000,-0.041,0.002,-0.143,0.114,-0.049,-0.030,0.201,-0.012,0.164,0.132,0.165,0.189,0.074,-0.046,0.009,-0.019,0.477,0.571,0.353,-0.008,-0.081,-0.036,-0.151,-0.108,-0.048,0.394,0.408,-0.008,0.042,-0.055,-0.066,-0.043,0.049,0.046,-0.0,0.410,0.338,0.384,-0.073,-0.018,0.079,-0.023,0.101,0.178
thickness,-0.001,0.026,0.070,0.087,0.365,-0.041,1.000,-0.053,-0.018,0.035,-0.059,0.055,-0.110,-0.050,-0.131,-0.211,-0.135,-0.125,-0.231,-0.024,-0.067,0.062,-0.011,-0.112,0.074,-0.012,-0.063,-0.031,0.107,0.120,0.377,0.134,0.062,-0.012,-0.043,-0.220,-0.173,0.000,0.118,0.219,-0.0,0.105,-0.185,0.126,0.358,0.005,0.165,0.020,0.166,-0.199
photo_soft_Chamber,0.326,0.013,0.011,0.018,0.003,0.002,-0.053,1.000,-0.024,0.004,0.020,0.071,-0.009,0.002,-0.002,-0.000,0.014,0.021,0.028,-0.062,0.019,-0.028,0.000,0.007,0.012,-0.020,0.029,0.004,0.016,0.051,-0.042,0.005,-0.019,-0.020,0.059,0.046,-0.007,0.021,-0.037,-0.030,-0.0,-0.014,-0.012,0.002,0.007,0.014,0.001,0.113,0.012,0.009
resist_target,-0.053,-0.049,0.140,0.151,-0.097,-0.143,-0.018,-0.024,1.000,-0.100,0.011,-0.079,-0.295,0.079,-0.324,-0.256,-0.349,-0.285,-0.312,0.092,-0.001,0.158,-0.073,-0.117,-0.015,0.049,0.300,0.152,0.234,0.071,-0.040,-0.151,-0.077,0.049,-0.014,0.070,0.071,0.009,-0.028,-0.177,0.0,-0.099,-0.113,-0.097,-0.254,0.023,0.282,-0.051,0.173,-0.364
N2_HMDS,0.004,0.009,-0.113,-0.111,0.161,0.114,0.035,0.004,-0.100,1.000,-0.072,0.026,0.037,-0.016,0.039,0.029,0.090,0.055,0.047,-0.009,-0.044,-0.007,0.013,0.071,0.019,-0.018,-0.127,-0.133,-0.115,-0.037,0.015,0.043,0.010,-0.018,-0.219,-0.242,-0.257,-0.012,0.089,0.097,0.0,0.009,0.090,0.055,0.213,0.041,-0.034,0.009,-0.026,0.073


In [84]:
df_list6 =  df_raw[[ 
    
        'Temp_implantation', 
        'Target']]

In [85]:
df_list6.corr()

,Temp_implantation,Target
Temp_implantation,1.000000,0.128536
Target,0.128536,1.000000


In [10]:
# ppm별로 나눌 필요=> dry(O2), wet(H2O) 별로 모델 나눔
#df_dummy=pd.get_dummies(df_raw[df_list_before])

In [11]:
# 데이터 나누기
df_x = df_raw[df_list_before]
df_y = df_raw['thickness']
df_train_x,df_test_x,df_train_y,df_test_y = train_test_split(df_x,df_y,test_size = 0.3,random_state = 1234)


In [28]:
para_leaf = [n_leaf * 1 for n_leaf in range(1,20)]
para_split = [n_split * 2 for n_split in range(2,20)]
para_depth = [depth for depth in range(1,11)]
estimator = DecisionTreeRegressor()
param_grid = {'criterion':["mse","friedman_mse",'mae'],"max_features":["auto","sqrt","log2"]
              ,"max_depth":para_depth,"min_samples_split":para_split, "min_samples_leaf":para_leaf}
grid_dt = GridSearchCV(estimator,param_grid,scoring="r2",n_jobs = -1)
grid_dt.fit(df_train_x,df_train_y)
print("best estimator model:\n{}".format(grid_dt.best_estimator_))
print("\nbest parameter:\n{}".format(grid_dt.best_params_))
print("\nbest score:\n{}".format(grid_dt.best_score_.round(3)))

KeyboardInterrupt: 